# PySCF Practice

In [3]:
from pyscf import gto, scf, dft
import numpy as np
from IPython.display import clear_output
import pandas as pd
from pathlib import Path

## Creaing a molecule
`gto.M` is a short cut function to initialize a molecule; you can set the basis or the symmetry

In [2]:
mol_h2o = gto.M(atom = 'O 0 0 0; H 0 1 0; H 0 0 1', basis = 'sto-3g')
mol_n3 = gto.M(atom = 'N 0 0 0; N 0 0 1; N 0 0 2', basis='sto-3g', charge=-1)
mol_n3_bent = gto.M(atom='N 0 0 0; N 0 1 1; N 0 1 0.5', basis='sto-3g', charge=-1)

`Mole` offers more customized intialisation. 

## Mean-Field theory
e.g. Hartree-fock (also called self-consistent field method (SCF))
### Package Options
- RHF (Restricted HF) - for closed-shell molecules with no unpaired e-. (idea: use the same equation to describe both electrons)
- UHF (Unrestricted HF) - for open-shell molecules, e.g. metals, radicals, ...
- ROHF (Restricted Open-Shell HF) - hybrid, keep the spin symmetry correct.
### Basis options

In [3]:
rhf_h2o = scf.RHF(mol_h2o)
e_h2o = rhf_h2o.kernel()

rhf_n3 = scf.RHF(mol_n3)
rhf_n3_bent = scf.RHF(mol_n3_bent)
e_n3 = rhf_n3.kernel()
e_n3_bent = rhf_n3_bent.kernel()

converged SCF energy = -74.9611711378677
converged SCF energy = -160.662076594209
converged SCF energy = -153.313769835815


Default spin is 0. For molecules with unpaired electrons, spin number must be updated; otherwise there will be error.

In [31]:
mol_no2 = gto.M(atom = 'N 0 0 0; O 0 1 0; O 0 0 1', basis='sto-3g', spin=1)
uhf_no2 = scf.UHF(mol_no2)
e_no2 = uhf_no2.kernel()


rohf_no2 = scf.ROHF(mol_no2)
e_no2_ro = rohf_no2.kernel()

converged SCF energy = -200.630088281403  <S^2> = 0.75238228  2S+1 = 2.0023809
converged SCF energy = -200.628674251033


For fun, here is a cubane molecule:

In [32]:
mol = gto.M(
    atom = '''
C   0.785   0.785   0.785
C  -0.785   0.785   0.785
C   0.785  -0.785   0.785
C   0.785   0.785  -0.785
C  -0.785  -0.785   0.785
C  -0.785   0.785  -0.785
C   0.785  -0.785  -0.785
C  -0.785  -0.785  -0.785
H   1.420   1.420   1.420
H  -1.420   1.420   1.420
H   1.420  -1.420   1.420
H   1.420   1.420  -1.420
H  -1.420  -1.420   1.420
H  -1.420   1.420  -1.420
H   1.420  -1.420  -1.420
H  -1.420  -1.420  -1.420
''',
    unit   = 'Angstrom',
    charge = 0,
    spin   = 0,
    basis  = 'sto-3g',   
)
rhf = scf.RHF(mol)
e = rhf.kernel()

converged SCF energy = -303.77968305976


### What exactly is it calculating here?


## Density Functional 
The same kinds of methods available:
- RKS
- UKS
- ROKS

The customisation here is **exchange-correlation** (what is that? and what is a 'basis'??)

In [36]:
from pyscf import dft
rks_h2o = dft.RKS(mol_h2o)
rks_h2o.xc = 'b3lyp'

e_rks = rks_h2o.kernel()

/home/yizhou_chen/miniconda3/envs/pyscf-env/lib/python3.12/site-packages/pyscf/dft/libxc.py:512: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -75.3186721647241


## A little on quantum mechanics
The average total energy for a state: $E[\Psi] = \int \Psi^* \hat{H} \Psi dr \equiv \left<\Psi | \hat{H} | \Psi \right> $

Note this notation is consistent with inner product notation: three items just means $\Psi*H*\Psi$

## Full Configuration Interaction

2. Add random noise to the 

In [32]:
from pyscf import gto, scf, cc

for r in (1.0, 1.1, 1.2):
    mol = gto.M(atom=f"H 0 0 0; H 0 0 {r}")
    mf = scf.RHF(mol).run()
    mycc = cc.CCSD(mf).run()
    print(mycc.e_tot)

converged SCF energy = -1.06610864931794
E(CCSD) = -1.101150330244479  E_corr = -0.03504168092654206
-1.1011503302444787
converged SCF energy = -1.03653887502918
E(CCSD) = -1.079192945018469  E_corr = -0.0426540699892894
-1.0791929450184694
converged SCF energy = -1.00510670656849
E(CCSD) = -1.056740746497147  E_corr = -0.05163403992865674
-1.0567407464971468
